# Using Vulkan

This notebook will take you through all the steps in using Vulkan to write policies.\
By the end, you will have:

1. Created a new policy, which can be used for online and batch evaluation,
2. Launched a run of your new policy,
3. Checked its results.

Let's dive in!

In [1]:
# Used to make our results easier to read
from pprint import pprint

# The actual Vulkan assets used
import vulkan_public.cli.client as vulkan
from vulkan_public.cli.context import Context

In [2]:
ctx = Context()

## Our First Policy

Let's start by creating a simple policy.\
It takes in a person's (fictional) credit score, and tells us whether that person would be approved or denied.

Later on, we'll see how to get data from an external data source and how to reuse parts of a policy.



### Policy and Policy Versions

In Vulkan, a `Policy` is an umbrella. It has a name, a description and an owner, and can have multiple versions.\
Each `Policy Version` can be used to make decisions separately, and you can have any number of them, making it easy to test and experiment.\
You can think of a Policy as how you make a decision on something, and a Version as each of the ways you'll want to try over time.

In [3]:
policy_id = vulkan.policy.create_policy(
    ctx,
    name="My First Policy",
    description="A nice policy description, which can be long.",
)

2024-11-28 15:35:36 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG Starting new HTTP connection (1): localhost:6001
2024-11-28 15:35:36 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /policies HTTP/1.1" 307 0
2024-11-28 15:35:37 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /policies/ HTTP/1.1" 200 77
2024-11-28 15:35:37 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Created policy My First Policy with id 5bc7f9c1-26ef-47b3-9685-f1554cb96ffa


Each `Policy Version` is defined by its code, the `repository_path` used to create it.\
It is totally independent of others versions, and can have its own definitions and dependencies.

In [4]:
policy_version_id = vulkan.policy.create_policy_version(
    ctx,
    policy_id=policy_id,
    version_name="v0.0.1",
    repository_path="../examples/policies/simple/",
)

2024-11-28 15:35:39 DESKTOP-GLELM79 vulkan_public.cli.context[82859] WARNING Failed to import module from entrypoint.
2024-11-28 15:35:39 DESKTOP-GLELM79 vulkan_public.cli.context[82859] WARNING Import-related issue from user code: No module named 'test_policy'
2024-11-28 15:35:39 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Creating workspace v0.0.1. This may take a while...
2024-11-28 15:36:44 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /policies/5bc7f9c1-26ef-47b3-9685-f1554cb96ffa/versions HTTP/1.1" 200 145
2024-11-28 15:36:44 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Created workspace v0.0.1 with policy version 990c9c5b-9d9b-47d1-b469-8a3ddbb9c777


### Runs: How we make decisions

Now that we've created a policy version, **we're ready to make our first decision**.

For that, we'll run our policy directly, giving it some input data.\
In this example, we want to return `APPROVED` if the score is greater than 400, and `DENIED` if not.

The user's data is passed to the `input_data` field. In our example, it's just a "score" value.\
We configure our threshold with the `config_variable` "SCORE_CUTOFF".

In [5]:
run_id, status = vulkan.run.trigger_run_by_policy_version_id(
    ctx,
    policy_version_id=policy_version_id,
    input_data={"tax_id": "1", "score": 600},
    config_variables={"SCORE_CUTOFF": 400},
)

run_data = vulkan.run.get_run_data(ctx, run_id)

# Retrieve the output of our decision.
# In the next section, we'll look into more details for the run data.
pprint(f'Output: {run_data["steps"]["branch_1"]["output"]}')

2024-11-28 15:36:44 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Lanching run with input data: {'tax_id': '1', 'score': 600}
2024-11-28 15:36:46 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /policy-versions/990c9c5b-9d9b-47d1-b469-8a3ddbb9c777/runs HTTP/1.1" 200 108
2024-11-28 15:36:47 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "GET /runs/39939483-7cca-476d-bcc7-5c7a57d28558 HTTP/1.1" 200 284
2024-11-28 15:36:48 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "GET /runs/39939483-7cca-476d-bcc7-5c7a57d28558 HTTP/1.1" 200 284
2024-11-28 15:36:53 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG Resetting dropped connection: localhost
2024-11-28 15:36:53 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "GET /runs/39939483-7cca-476d-bcc7-5c7a57d28558 HTTP/1.1" 200 290
2024-11-28 15:36:54 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "GE

'Output: approved'


#### What happened?

When you trigger a Run using Vulkan, your code will be executed.\
The *actual* execution is happening remotely, on Vulkan's servers.\
It uses the code you uploaded for the `Policy Version`. \
This way, we can handle the infrastructure for you, allowing simple code to scale.

The engine will receive your inputs, the configuration you want to use, and run your code, producing a detailed log of each step.\
Let's take a look at the entire run data object returned.

You'll see that it has a couple of important parts:

1. Run Metadata:
    1.  Run ID: An unique identifier for this run in Vulkan. You can use this to see logs, outputs, or to visualize the decision on the web app.
    2.  Status: Whether or not the run finished successfully.
    3.  Last Updated At: A timestamp for when the run finished.
2. Run Steps:
    1. Each step is identified by the name we gave it, in this case "approved" and "branch";
    2. For each step, we have (1) `output`, which contains the actual output of the step, and (2) `metadata`, which brings information on what kind of step this was, how long it took, etc.

In [6]:
pprint(run_data)

{'last_updated_at': '2024-11-28T18:36:50.228568Z',
 'run_id': '39939483-7cca-476d-bcc7-5c7a57d28558',
 'status': 'SUCCESS',
 'steps': {'approved': {'metadata': {'end_time': 1732819010.2492735,
                                     'error': None,
                                     'extra': None,
                                     'node_type': 'TERMINATE',
                                     'start_time': 1732819010.214999,
                                     'step_name': 'approved'},
                        'output': 'APPROVED'},
           'branch_1': {'metadata': {'end_time': 1732819009.1410248,
                                     'error': None,
                                     'extra': {'choices': ['approved',
                                                           'denied']},
                                     'node_type': 'BRANCH',
                                     'start_time': 1732819009.1188374,
                                     'step_name': 'branch_1'},
   

## Convenience features for Policies

### Setting default values for config variables

You can specify default values for your Config Variables in each version.\
These are useful when you've already decided on the values and want to have them for all runs.

In [7]:
vulkan.policy_version.set_variables(
    ctx,
    policy_version_id=policy_version_id,
    variables=[{"name": "SCORE_CUTOFF", "value": "500"}],
)

2024-11-28 15:37:57 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Setting variables: [{'name': 'SCORE_CUTOFF', 'value': '500'}]
2024-11-28 15:37:57 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG Resetting dropped connection: localhost
2024-11-28 15:37:57 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "PUT /policy-versions/990c9c5b-9d9b-47d1-b469-8a3ddbb9c777/variables HTTP/1.1" 200 112


{'policy_version_id': '990c9c5b-9d9b-47d1-b469-8a3ddbb9c777',
 'variables': [{'name': 'SCORE_CUTOFF', 'value': '500'}]}

### Marking a version as "Active"

In many cases, we want to have a standard version for a policy.\
This is useful for consumers of this policy: they can just get the current version of "My First Policy", no matter what that is.

In [8]:
vulkan.policy.set_active_version(
    ctx, 
    policy_id=policy_id, 
    policy_version_id=policy_version_id,
)

2024-11-28 15:38:00 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "PUT /policies/5bc7f9c1-26ef-47b3-9685-f1554cb96ffa HTTP/1.1" 200 118


In [9]:
run_id, status = vulkan.run.trigger_run_by_policy_id(
    ctx, 
    policy_id=policy_id, 
    input_data={"tax_id": "1", "score": 600}
)

run_data = vulkan.run.get_run_data(ctx, run_id)
pprint(run_data)

2024-11-28 15:38:00 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Lanching run with input data: {'tax_id': '1', 'score': 600}
2024-11-28 15:38:02 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /policies/5bc7f9c1-26ef-47b3-9685-f1554cb96ffa/runs HTTP/1.1" 200 100
2024-11-28 15:38:03 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "GET /runs/90dd0437-7bc0-49ed-9b75-d2969f6c2245 HTTP/1.1" 200 284
2024-11-28 15:38:04 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "GET /runs/90dd0437-7bc0-49ed-9b75-d2969f6c2245 HTTP/1.1" 200 284
2024-11-28 15:38:09 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG Resetting dropped connection: localhost
2024-11-28 15:38:09 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "GET /runs/90dd0437-7bc0-49ed-9b75-d2969f6c2245 HTTP/1.1" 200 282
2024-11-28 15:38:10 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "GET /runs

{'last_updated_at': '2024-11-28T18:38:04.507979Z',
 'run_id': '90dd0437-7bc0-49ed-9b75-d2969f6c2245',
 'status': 'FAILURE',
 'steps': {}}


## Components: What, Why and How?

### Creating a couple of Components

In [10]:
serasa_component_id = vulkan.component.create_component(ctx, name="serasa_component")
serasa_comp_version = vulkan.component.create_component_version(
    ctx,
    component_id=serasa_component_id,
    version_name="v0.0.1",
    repository_path="../examples/components/serasa/",
)

2024-11-28 15:38:18 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG Resetting dropped connection: localhost
2024-11-28 15:38:19 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /components HTTP/1.1" 307 0
2024-11-28 15:38:20 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /components/ HTTP/1.1" 200 193
2024-11-28 15:38:20 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Created component serasa_component with id 4b707325-9f63-4cef-a8ff-40901ad12227
2024-11-28 15:38:22 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /components/4b707325-9f63-4cef-a8ff-40901ad12227/versions HTTP/1.1" 200 63
2024-11-28 15:38:22 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Created component version be50fe5b-3853-4751-b66f-c563fa36808d (v0.0.1) for component 4b707325-9f63-4cef-a8ff-40901ad12227.


In [11]:
scr_component_id = vulkan.component.create_component(ctx, name="scr_component")
scr_comp_version = vulkan.component.create_component_version(
    ctx,
    component_id=scr_component_id,
    version_name="v0.0.1",
    repository_path="../examples/components/scr_component/",
)

2024-11-28 15:38:22 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /components HTTP/1.1" 307 0
2024-11-28 15:38:23 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /components/ HTTP/1.1" 200 190
2024-11-28 15:38:23 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Created component scr_component with id 9ef11447-425b-45f5-a1e6-75647ffeb60f
2024-11-28 15:38:25 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /components/9ef11447-425b-45f5-a1e6-75647ffeb60f/versions HTTP/1.1" 200 63
2024-11-28 15:38:25 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Created component version 58b95ecc-25e4-4c20-a549-be85058f5b38 (v0.0.1) for component 9ef11447-425b-45f5-a1e6-75647ffeb60f.


### Creating a Policy that uses Components

In [12]:
policy_id = vulkan.policy.create_policy(
    ctx,
    name="Test Policy",
    description="Test Policy Description",
)

2024-11-28 15:38:26 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /policies HTTP/1.1" 307 0
2024-11-28 15:38:26 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /policies/ HTTP/1.1" 200 73
2024-11-28 15:38:26 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Created policy Test Policy with id 1f22378c-ea26-493a-a296-0aab95ac22e1


In [13]:
policy_version_id = vulkan.policy.create_policy_version(
    ctx,
    policy_id=policy_id,
    version_name="v0.0.1",
    repository_path="../examples/policies/onboarding/",
)

2024-11-28 15:38:30 DESKTOP-GLELM79 vulkan_public.cli.context[82859] WARNING Failed to import module from entrypoint.
2024-11-28 15:38:30 DESKTOP-GLELM79 vulkan_public.cli.context[82859] WARNING Import-related issue from user code: No module named 'onboarding'
2024-11-28 15:38:30 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Creating workspace v0.0.1. This may take a while...
2024-11-28 15:39:36 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /policies/1f22378c-ea26-493a-a296-0aab95ac22e1/versions HTTP/1.1" 200 145
2024-11-28 15:39:36 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Created workspace v0.0.1 with policy version 8cb18ea2-0937-438d-85e4-327ac932c1bd


In [14]:
vulkan.run.trigger_run_by_policy_version_id(
    ctx,
    policy_version_id=policy_version_id,
    input_data={"cpf": "1"},
    config_variables={"CORTE_SERASA": "400"},
)

2024-11-28 15:39:36 DESKTOP-GLELM79 vulkan_public.cli.context[82859] INFO Lanching run with input data: {'cpf': '1'}
2024-11-28 15:39:39 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "POST /policy-versions/8cb18ea2-0937-438d-85e4-327ac932c1bd/runs HTTP/1.1" 200 108
2024-11-28 15:39:39 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "GET /runs/44b6b535-bd3e-4731-8439-1295dfc50c83 HTTP/1.1" 200 284
2024-11-28 15:39:40 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "GET /runs/44b6b535-bd3e-4731-8439-1295dfc50c83 HTTP/1.1" 200 284
2024-11-28 15:39:45 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG Resetting dropped connection: localhost
2024-11-28 15:39:46 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG http://localhost:6001 "GET /runs/44b6b535-bd3e-4731-8439-1295dfc50c83 HTTP/1.1" 200 284
2024-11-28 15:39:51 DESKTOP-GLELM79 urllib3.connectionpool[82859] DEBUG Resetting dropped connection: localhost
20

('44b6b535-bd3e-4731-8439-1295dfc50c83', True)